In [9]:
!pip install wget

Looking in indexes: https://pypi.tuna.tsinghua.edu.cn/simple

[notice] A new release of pip available: 22.1.2 -> 22.2.2
[notice] To update, run: pip install --upgrade pip


In [10]:
!pip install gensim

Looking in indexes: https://pypi.tuna.tsinghua.edu.cn/simple

[notice] A new release of pip available: 22.1.2 -> 22.2.2
[notice] To update, run: pip install --upgrade pip


In [11]:
from paddle.io import IterableDataset
from paddledata.datapipes.iter import *
from paddledata.extension import *
import os
import pandas as pd
import numpy as np
import jieba
from gensim.models import word2vec

In [12]:
class RandomIterableDataset(IterableDataset):
    def __init__(self, x, y):
        self.x = x
        self.y = y

    def __iter__(self):
        for i in range(len(self.x)):
            data = self.x[i]
            label = self.y[i]
            yield data, label

x = np.random.random([64, 3]).astype('float32')
y = np.random.randint(0, 2, (64, 1)).astype('int64')
iterable_dataset = RandomIterableDataset(x, y)

for data, label in iterable_dataset:
    print(data, label)

[0.41573614 0.1466568  0.872243  ] [0]
[0.2451987  0.46963426 0.13936372] [0]
[0.49954247 0.16031009 0.14504197] [1]
[0.51358306 0.6444739  0.687803  ] [0]
[0.75018185 0.6686349  0.29817727] [1]
[0.64661837 0.91101116 0.16727293] [1]
[0.22724913 0.44764566 0.897494  ] [0]
[0.7714584  0.9453056  0.91553456] [0]
[0.29458395 0.0442587  0.17968315] [0]
[0.10128532 0.41836375 0.4047683 ] [1]
[0.58012176 0.09377854 0.48462287] [1]
[0.67806876 0.28449148 0.9118643 ] [1]
[0.19493045 0.32390922 0.9695272 ] [0]
[0.8050516  0.00169936 0.23770203] [0]
[0.35482702 0.4070339  0.97700566] [0]
[0.17162652 0.08129493 0.74695426] [0]
[0.72566366 0.4953351  0.54041314] [0]
[0.3508272  0.45217577 0.18412499] [1]
[0.7463199  0.7102503  0.20235571] [1]
[0.8987816  0.65358114 0.4779008 ] [0]
[0.0301952  0.5476607  0.27981567] [1]
[0.15650262 0.5045874  0.3188929 ] [0]
[0.90891504 0.71147734 0.8910655 ] [0]
[0.48175022 0.85393786 0.8183875 ] [0]
[0.19307536 0.40724182 0.81041837] [0]
[0.32431147 0.20524348 0.

In [13]:
Pipelines = list()
Pipelines.append(IterableMapper(map_func=lambda x: x + 1))
Pipelines.append(IterableFilter(predicate=lambda x: x[1] > 0.5))
Pipelines.append(IterableShuffler())
Pipelines.append(IterableBatcher(batch_size=5, drop_last=False))
processing = IterableDataPipes(pipelines=Pipelines)

result = processing(iterable_dataset)
for data, label in result:
    print(data, label)

(array([1.3508272, 1.4521757, 1.184125 ], dtype=float32), array([1.9368267, 1.6837218, 1.5044799], dtype=float32), array([1.6466184, 1.9110112, 1.1672729], dtype=float32), array([1.7780354, 1.0661376, 1.8922435], dtype=float32), array([1.7501819, 1.6686349, 1.2981772], dtype=float32)) (array([1]), array([1]), array([1]), array([1]), array([1]))
(array([1.1012853, 1.4183638, 1.4047682], dtype=float32), array([1.4426974, 1.4931389, 1.8083199], dtype=float32), array([1.5801218, 1.0937785, 1.4846228], dtype=float32), array([1.4995425, 1.16031  , 1.145042 ], dtype=float32), array([1.0278617, 1.468683 , 1.899376 ], dtype=float32)) (array([1]), array([1]), array([1]), array([1]), array([1]))
(array([1.9239603, 1.3832438, 1.229795 ], dtype=float32), array([1.1536553, 1.5580759, 1.2059621], dtype=float32), array([1.724664 , 1.3188868, 1.7400224], dtype=float32), array([1.1957374, 1.9440678, 1.4120699], dtype=float32), array([1.5406442, 1.7694263, 1.805318 ], dtype=float32)) (array([1]), array([

In [14]:
class MyDataset(IterableDataset):
    def __init__(self, paths):
        self.paths = paths
    
    def __iter__(self):
        for path in self.paths:
            yield path

paths = ['https://bj.bcebos.com/v1/ai-studio-online/7f619988564f4a7287440a7789bfa8e7e88bcf33f21b4bea862dc73491258ff9?responseContentDisposition=attachment%3B%20filename%3Dtest.zip&authorization=bce-auth-v1%2F5cfe9a5e1454405eb2a975c43eace6ec%2F2022-09-25T07%3A01%3A41Z%2F-1%2F%2F5ad9522f9d84765641f4c316cce0474bcd76c92991407442f7bd657f9cd59a9e']
example = MyDataset(paths)
for i in example:
    print(i)

https://bj.bcebos.com/v1/ai-studio-online/7f619988564f4a7287440a7789bfa8e7e88bcf33f21b4bea862dc73491258ff9?responseContentDisposition=attachment%3B%20filename%3Dtest.zip&authorization=bce-auth-v1%2F5cfe9a5e1454405eb2a975c43eace6ec%2F2022-09-25T07%3A01%3A41Z%2F-1%2F%2F5ad9522f9d84765641f4c316cce0474bcd76c92991407442f7bd657f9cd59a9e


In [15]:
Pipelines = list()
# 远程文件操作
Pipelines.append(Downloader(save_path='data/example'))
# 文件解压
Pipelines.append(Extractor(save_dir='./data/example/decompression'))
processing = DataPipes(pipelines=Pipelines)
final_dataset = processing(example)
for temp in final_dataset:
    print(temp)

/home/aistudio/work/data/example/decompression/Sina_News.txt
/home/aistudio/work/data/example/decompression/California_Gull.jpg
/home/aistudio/work/data/example/decompression/Herring_Gull.jpg
/home/aistudio/work/data/example/decompression/Groove_Billed_Ani.jpg


In [16]:
class MyDataset(IterableDataset):
    def __init__(self, paths):
        self.paths = paths
    
    def __iter__(self):
        for path in self.paths:
            yield path

paths = ['work/data/example/decompression/Sina_News.txt']
example = MyDataset(paths)
for i in example:
    print(i)

work/data/example/decompression/Sina_News.txt


In [17]:
final = iter(final_dataset)

In [18]:
print(next(final))

/home/aistudio/work/data/example/decompression/Sina_News.txt
